#### Practice 3: YOLOv3

In [ ]:
import cv2
import numpy as np

# Load YOLO
net =  cv2.dnn.readNet('cfg/yolov3.weights', 'cfg/yolov3.cfg')
classes = []
with open('cfg/coco.names', 'r') as f:
    classes = f.read().splitlines()

# Load input image
image = cv2.imread('images/lake.jpg')
height, width, _ = image.shape

# Preprocess input image
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)

